## Importing libraries

In [1]:
import textract
import pandas as pd
import os
import json

### Folder paths

In [2]:
resume_folder_path = '/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/Eximius Employees Resumes PDF Format'
skillsheet_file_path = '/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/Key Word Search.xlsx'

In [3]:
def get_all_skills(skillsheet_file_path):
    skill_sheet = pd.read_excel(skillsheet_file_path, header=[1, 2], dtype=str)
    skill_sheet = skill_sheet.drop(skill_sheet.columns[0], axis=1)
    skills_dict = {}
    all_skills = []
    for label, label_data in skill_sheet.iteritems():
        skills_dict[label] = label_data.dropna()
        all_skills.extend(label_data.dropna().tolist())
        
    return all_skills

In [4]:
all_skills = get_all_skills(skillsheet_file_path)

In [5]:
def extract_yoe(rid):
    temp_yoe = rid.split('/')[-1].split('_')
    yoe = 0
    if temp_yoe[0].isdigit():
        if len(temp_yoe) == 2:
            yoe = temp_yoe[0]+'.'+temp_yoe[1]
        else:
            yoe = temp_yoe[0]
    else:
        yoe = 0
        
    return yoe


In [6]:
def create_resume_data_json(resume_folder_path):
    all_resume_data = []
    all_resume_string = []
    xy = []
    cnt = 0
    for each_file in os.listdir(resume_folder_path):
        each_resume_dict = {}
        each_file_path = os.path.join(resume_folder_path, each_file)
        each_file_str = textract.process(each_file_path, encoding='ascii').decode()

        each_resume_dict['Filename'] = each_file
        each_resume_dict['ResumeString'] = each_file_str
        skills_found = [each_skill.strip() for each_skill in all_skills if each_skill.strip() in each_file_str]
        each_resume_dict['SkillsFound'] = list(set(skills_found))

        rid = each_file_str.split('\n')[0]

        each_resume_dict['YearsOfExperience'] = float(extract_yoe(rid))
        engineerDesc = each_file_str.split('\n')[1].replace('-', '').split('  ')[0]
        each_resume_dict['engineerType'] = engineerDesc
        all_resume_string.append(each_file_str)
        all_resume_data.append(each_resume_dict)
        cnt += 1
        
    with open('/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/resume_data.json', 'w') as f:
        json.dump(all_resume_data, f, indent=4)

In [7]:
create_resume_data_json(resume_folder_path)

In [8]:
resume_data = json.load(open('/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/resume_data.json', 'r'))

In [9]:
resume_data_df = pd.DataFrame(resume_data)

In [10]:
resume_data_df['YearsOfExperience'] = resume_data_df['YearsOfExperience'].astype('float')

In [11]:
def filtering_by_skills(skills_query, resume_data_df):
    all_skill_cnt, all_matched_skills = [], []
    for each_skill_row in resume_data_df['SkillsFound']:
        
        skill_cnt = 0
        skills_matched = []
        for each_query_skill in skills_query.split(','):
            
            each_query_skill = each_query_skill.strip()
            
            if each_query_skill in each_skill_row:
                skill_cnt += 1
                skills_matched.append(each_query_skill)
                
        all_skill_cnt.append(skill_cnt)
        all_matched_skills.append(skills_matched)
                
                
    resume_data_df['MatchedSkillsCount'] = all_skill_cnt
    resume_data_df['MatchedSkills'] = all_matched_skills
    
    
    return resume_data_df[resume_data_df.MatchedSkillsCount>0].sort_values(by='MatchedSkillsCount', ascending=False)

In [12]:
def filtering_by_experience(exp, operation, filtered_by_skills_df):
    
    query_op = 'filtered_by_skills_df.YearsOfExperience' + operation + str(exp)
    
    return filtered_by_skills_df[eval(query_op)]

In [13]:
user_query = 'Embedded C, Android'
# user_query = 'Embedded C, Android, Firmware, BSP, SATA, CAN, Python, Autosar, IPC, Framework'

In [14]:
filtered_by_skills_df = filtering_by_skills(user_query, resume_data_df)

In [15]:
filtering_by_experience(4, '>', filtered_by_skills_df)

,Filename,ResumeString,SkillsFound,YearsOfExperience,engineerType,MatchedSkillsCount,MatchedSkills
155,Eximius Design - Pavankumar K Arakere.pdf,ED/DE/596/PKA/6\nDevelopment Engineer Pavanku...,"[Microcontrollers, 802.11, I2C, Automation, C,...",6.0,Development Engineer,2,"[Embedded C, Android]"
163,Eximius Design - Naveen Kumar Ginkala.pdf,ED/DE/467/GNK/8_5\nDevelopment Engineer G Nav...,"[RTOS, C, Android, Python, Embedded C, Firmware]",8.5,Development Engineer,2,"[Embedded C, Android]"
143,Eximius Design - Sibananda Sahu.pdf,ED/DE/881/SS/8\nDevelopment Engineer Sibanand...,"[802.11, I2C, RTOS, C, SATA, Android, Device d...",8.0,Development Engineer,2,"[Embedded C, Android]"
38,Eximius Design - Shravan Kumar Kondi.pdf,ED/DE/680/SKK/4_3\nDevelopment Engineer Shrav...,"[I2C, Automation, C, Android, BSP, Embedded C,...",4.3,Development Engineer,2,"[Embedded C, Android]"
136,Eximius Design - Sudhir Kumar Reddy.pdf,ED/DE/275/SKRC/7_5\nDevelopment Engineer Sudh...,"[C, SATA, Android, Embedded C, Firmware, SSD, ...",7.5,Development Engineer,2,"[Embedded C, Android]"
28,Eximius Design - Nagelly Arunkumar.pdf,ED/DE/372/NAK/4_5\nDevelopment Engineer Nagel...,"[RTOS, C, Android, Python, IPC, Embedded C, Fr...",4.5,Development Engineer,2,"[Embedded C, Android]"
187,Eximius Design - Anbu Manikandan C.pdf,ED/DE/675/AMC/10\nDevelopment Engineer Anbu M...,"[RTOS, C, Android, ARM, MSM, Python, Embedded ...",10.0,Development Engineer,2,"[Embedded C, Android]"
159,Eximius Design - Nandini Krishna Nemalapudi.pdf,ED/VA/243/NKN/10_5\nValidation Engineer - Nand...,"[802.11, RTOS, C, Android, MSM, Python, BCM, E...",10.5,Validation Engineer,2,"[Embedded C, Android]"
5,Eximius Design - Rahul Kumar Choube.pdf,ED/DE/536/RKC/8\nDevelopment Engineer Rahul K...,"[Microcontrollers, I2C, RTOS, C, Yocto, Androi...",8.0,Development Engineer,2,"[Embedded C, Android]"
103,Eximius Design - Arun T.pdf,ED/DE/889/TA/9_6\nDevelopment Engineer T Arun...,"[I2C, C, Android, MSM, BSP, BCM, UART, CAN, FP...",9.6,Development Engineer,2,"[Embedded C, Android]"
